# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

## Web-scraping

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с реального сайта [nplus1.ru](https://nplus1.ru/).

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [11]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [12]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [13]:
page  # response 200 - страница загружена

<Response [200]>

In [4]:
# page.text

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [14]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [15]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [7]:
# soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [8]:
# print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).


Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [16]:
for link in soup.find_all('a'):
    print(link.get('href'))
#     break

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https://nplus1.ru/search?tags=874
https://nplus1.ru/search?tags=880
https://nplus1.ru/search?tags=768
https://nplus1.ru/search?tags=890
https://nplus1.ru/search?tags=871
https://nplus1.ru/search?tags=876
https://nplus1.ru/search?tags=775
https://nplus1.ru/search?tags=767
https://nplus1.ru/search?tags=771
https://nplus1.ru/search?tags=772
https://nplus1.ru/search?tags=778
https://nplus1.ru/search?tags=917
https://nplus1.ru/search?tags=918
https://nplus1.ru/search?tags=824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [7]:
# urls = []

# for link in soup.find_all('a'):
#     if '/news' in link.get('href'):
#         urls.append('https://nplus1.ru'+link.get('href'))

urls = [link.get('href') 
        for link in soup.find_all('a') 
        if 'https://nplus1.ru/news/2025' in link.get('href')]

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки (название сайта) отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [17]:
urls[:5]

['https://nplus1.ru/news/2025/04/02/social-memory-effect',
 'https://nplus1.ru/news/2025/04/05/fram-2-end',
 'https://nplus1.ru/news/2025/04/04/falcaria-bilineata',
 'https://nplus1.ru/news/2025/04/04/b14-static-fire-test-before-lift9',
 'https://nplus1.ru/news/2025/04/04/chinese-carbon-sinks']

In [18]:
full_urls = []

for u in urls:
    full_urls.append(u) 

full_urls

['https://nplus1.ru/news/2025/04/02/social-memory-effect',
 'https://nplus1.ru/news/2025/04/05/fram-2-end',
 'https://nplus1.ru/news/2025/04/04/falcaria-bilineata',
 'https://nplus1.ru/news/2025/04/04/b14-static-fire-test-before-lift9',
 'https://nplus1.ru/news/2025/04/04/chinese-carbon-sinks',
 'https://nplus1.ru/news/2025/04/04/mass-grave-of-roman-soldiers',
 'https://nplus1.ru/news/2025/04/04/burial-of-buturlin',
 'https://nplus1.ru/news/2025/04/04/enjoy-your-drink',
 'https://nplus1.ru/news/2025/04/04/unfortunately-it-happens',
 'https://nplus1.ru/news/2025/04/03/wealth-longlife',
 'https://nplus1.ru/news/2025/04/03/ancient-dna-from-the-green-sahara',
 'https://nplus1.ru/news/2025/04/03/southern-ocean-warming',
 'https://nplus1.ru/news/2025/04/03/triple-std-detected',
 'https://nplus1.ru/news/2025/04/03/enjoy-it-nevermore',
 'https://nplus1.ru/news/2025/04/02/lithium-triangle',
 'https://nplus1.ru/news/2025/04/02/shingles-vaccine-vs-dementia',
 'https://nplus1.ru/news/2025/04/02/or

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [19]:
url0 = urls[0]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')

In [20]:
url0

'https://nplus1.ru/news/2025/04/02/social-memory-effect'

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [21]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Механическая клешня не помогла человекообразным приматам запомнить объекты" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/1006260/67ed64685633f_cover_share.png" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2025/04/02/social-memory-effect" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [22]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Катерина Петрова" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [23]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Катерина Петрова" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [24]:
soup0.find_all('meta', {'name' : 'author'})[0].get('content')

'Катерина Петрова'

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [25]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs
author

{'name': 'author', 'content': 'Катерина Петрова'}

Аналогичным образом извлечем дату, заголовок и описание.

In [26]:
soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

'Механическая клешня не\xa0помогла человекообразным приматам запомнить объекты'

In [27]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
description = soup0.find_all('h1', {'class':'text-34 md:text-42 xl:text-52 break-words'})[0].get_text().strip() \
+ '\n' + soup0.find_all('p', {'class':"text-36 md:text-44 xl:text-54 font-spectral text-main-gray mb-6"})[0].get_text().strip()



In [28]:
description

'Механическая клешня не\xa0помогла человекообразным приматам запомнить объекты\nНо\xa0взрослые обезьяны запомнили башни, построенные человеческой рукой'

Осталось вытащить рубрики и сложность текста. Если мы посмотрим на исходный код страницы, мы увидим, что нужная нам информация находится в тэгах `<p></p>`:

In [29]:
new_data = soup0.find_all('div', {'class':"flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})[0].find_all('span')[1:]

time_published = new_data[0].get_text()
date_published = new_data[1].get_text()
complexity = new_data[2].get_text()
themes = ', '.join(i.get_text() for i in new_data[3:])

In [30]:
new_data

[<span class="group-hover:text-main transition-colors duration-75">19:24</span>,
 <span class="group-hover:text-main transition-colors duration-75">02.04.25</span>,
 <span class="group-hover:text-main transition-colors duration-75">2.7</span>,
 <span class="group-hover:text-main transition-colors duration-75">Зоология</span>]

In [31]:
paragraphs = soup0.find_all('p', {'class': 'mb-6'})[1:]


In [32]:
paragraphs

[<p class="mb-6">Приматологи из Австралии, Германии и США обнаружили, что только взрослые приматы лучше запоминают предметы, с которыми взаимодействует человек, а не механическая клешня. Этот эффект они объяснили повышенным вниманием к социальному контексту, на который указывала пониженная частота сердечных сокращений. Исследование <a href="https://www.sciencedirect.com/science/article/pii/S0003347225000089?via=ihub%20https://www.sciencedirect.com/science/article/pii/S0003347225000089?via=ihub" rel="noreferrer noopener" target="_blank">опубликовано</a> в журнале <em>Animal Behaviour</em>.</p>,
 <p class="mb-6">2 апреля отмечается Всемирный день информирования об аутизме, поэтому сегодня новости на <em>N + 1</em> выходят с иллюстрациями от студентов «Антон тут рядом». Этот благотворительный фонд уже больше десяти лет помогает людям с расстройствами аутистического спектра в России</p>,
 <p class="mb-6">Жизнь в группах требует особой чувствительности к социальным стимулам — и многие прима

In [33]:
links = set()
for p in paragraphs:
    links|={i.get('href') for i in p.find_all('a')}
links = ', '.join(links)

In [34]:
full_text = '\n'.join([p.get_text() for p in paragraphs])

Не прошло и двух пар, как мы разобрались со всем :) Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [35]:
def GetNews(url0):
    """
    Returns a tuple with 
    url0, date_published, time_published, author, description, title, complexity, themes, links,full_text
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
    description = soup0.find_all('h1', {'class':'text-34 md:text-42 xl:text-52 break-words'})[0].get_text().strip() \
+ '\n' + soup0.find_all('p', {'class':"text-36 md:text-44 xl:text-54 font-spectral text-main-gray mb-6"})[0].get_text().strip()


    
    new_data = soup0.find_all('div', {'class':"flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})[0].find_all('span')[1:]

    time_published = new_data[0].get_text()
    date_published = new_data[1].get_text()
    complexity = new_data[2].get_text()
    themes = ', '.join(i.get_text() for i in new_data[3:])
    
    paragraphs = soup0.find_all('p', {'class': 'mb-6'})[1:]
    links = set()
    for p in paragraphs:
        links|={i.get('href') for i in p.find_all('a')}
    links -= {None} # Если вдруг у нас есть None-ссылка
    links = ', '.join(links)
    full_text = '\n'.join([p.get_text() for p in paragraphs])
    
    return url0, date_published, time_published, author, description, title, complexity, themes, links,full_text

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [36]:
from time import sleep
from tqdm import tqdm

In [37]:
import random

In [38]:
urls

['https://nplus1.ru/news/2025/04/02/social-memory-effect',
 'https://nplus1.ru/news/2025/04/05/fram-2-end',
 'https://nplus1.ru/news/2025/04/04/falcaria-bilineata',
 'https://nplus1.ru/news/2025/04/04/b14-static-fire-test-before-lift9',
 'https://nplus1.ru/news/2025/04/04/chinese-carbon-sinks',
 'https://nplus1.ru/news/2025/04/04/mass-grave-of-roman-soldiers',
 'https://nplus1.ru/news/2025/04/04/burial-of-buturlin',
 'https://nplus1.ru/news/2025/04/04/enjoy-your-drink',
 'https://nplus1.ru/news/2025/04/04/unfortunately-it-happens',
 'https://nplus1.ru/news/2025/04/03/wealth-longlife',
 'https://nplus1.ru/news/2025/04/03/ancient-dna-from-the-green-sahara',
 'https://nplus1.ru/news/2025/04/03/southern-ocean-warming',
 'https://nplus1.ru/news/2025/04/03/triple-std-detected',
 'https://nplus1.ru/news/2025/04/03/enjoy-it-nevermore',
 'https://nplus1.ru/news/2025/04/02/lithium-triangle',
 'https://nplus1.ru/news/2025/04/02/shingles-vaccine-vs-dementia',
 'https://nplus1.ru/news/2025/04/02/or

In [39]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in tqdm(urls):
    try:
        res = GetNews(link)
        news.append(res)
        sleep(random.random()) # задержка в 3 секунды
    except:
        print('Ссылка битая')

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:40<00:00,  1.25it/s]


In [40]:
urls[40]

'https://nplus1.ru/news/2025/03/20/supersolidity-in-photonic-crystal'

In [41]:
page0 = requests.get(urls[40])
soup0 = BeautifulSoup(page0.text, 'lxml')

In [42]:
paragraphs = soup0.find_all('p', {'class': 'mb-6'})[1:]

In [43]:
links = set()
for p in paragraphs:
    links|={i.get('href') for i in p.find_all('a')}

In [44]:
links - {None}

{'https://nplus1.ru/material/2024/07/26/polariton-transistors',
 'https://nplus1.ru/news/2017/03/02/supersolid',
 'https://nplus1.ru/news/2019/08/05/supersolid-excitations',
 'https://www.nature.com/articles/s41586-025-08616-9'}

Так теперь выглядит первый элемент списка:

In [45]:
news[0]

('https://nplus1.ru/news/2025/04/02/social-memory-effect',
 '02.04.25',
 '19:24',
 'Катерина Петрова',
 'Механическая клешня не\xa0помогла человекообразным приматам запомнить объекты\nНо\xa0взрослые обезьяны запомнили башни, построенные человеческой рукой',
 'Механическая клешня не\xa0помогла человекообразным приматам запомнить объекты',
 '2.7',
 'Зоология',
 'https://nplus1.ru/news/2018/09/12/chimps-vs-toddlers, https://www.cell.com/current-biology/fulltext/S0960-9822(05)00093-X?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS096098220500093X%3Fshowall%3Dtrue, https://www.sciencedirect.com/science/article/pii/S0003347225000089?via=ihub%20https://www.sciencedirect.com/science/article/pii/S0003347225000089?via=ihub, https://www.nature.com/articles/srep40926',
 'Приматологи из\xa0Австралии, Германии и\xa0США обнаружили, что только взрослые приматы лучше запоминают предметы, с\xa0которыми взаимодействует человек, а\xa0не\xa0механическая клешня. Этот эффект они объясни

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [46]:
import pandas as pd

In [47]:
df = pd.DataFrame(news)

In [48]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9
0,https://nplus1.ru/news/2025/04/02/social-memor...,02.04.25,19:24,Катерина Петрова,Механическая клешня не помогла человекообразны...,Механическая клешня не помогла человекообразны...,2.7,Зоология,https://nplus1.ru/news/2018/09/12/chimps-vs-to...,"Приматологи из Австралии, Германии и США обнар..."
1,https://nplus1.ru/news/2025/04/05/fram-2-end,05.04.25,23:35,Александр Войтюк,Завершился первый частный космический полет на...,Завершился первый частный космический полет на...,1.9,Космонавтика,https://spacenews.com/fram2-completes-polar-or...,Частный космический полет Fram2 завершился дне...


Переименуем столбцы в базе.

In [49]:
df.columns = ['url', 'date_published', 'time_published', 'author', 'description', 'title', 'complexity', 'themes', 'links','full_text']

Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [50]:
df['complexity'] = df['complexity'].apply(float)

Теперь сложность представлена в базе как количественный показатель, и описывать ее можно соответствующим образом:

In [51]:
df.complexity.describe()

count    50.000000
mean      3.362000
std       1.871253
min       1.100000
25%       1.900000
50%       2.850000
75%       4.500000
max       9.100000
Name: complexity, dtype: float64

In [52]:
from matplotlib.cbook import flatten

In [53]:
sorted(set(flatten([i.split(', ') for i in df.themes.unique()])))

['Антропология',
 'Археология',
 'Астрономия',
 'Биология',
 'Гаджеты',
 'Зоология',
 'История',
 'Космонавтика',
 'Математика',
 'Медицина',
 'Палеонтология',
 'Психология',
 'Роботы и дроны',
 'Социология',
 'Физика',
 'Химия',
 'Экология и климат']

In [54]:
df[df.themes.apply(lambda x: 'зоология' in x.lower())].sample(1).full_text.values[0]

'Герпетологи пришли к\xa0выводу, что предки игуан из\xa0рода Brachylophus, которые населяют архипелаги Фиджи и\xa0Тонга, попали сюда из\xa0Северной Америки, преодолев более 8000 километров по\xa0Тихому океану на\xa0переносимом течениями растительном плоту. На\xa0это указал анализ эволюционных связей, согласно которому ближайшими родственниками фиджийских и\xa0тонганских игуан являются пустынные игуаны с\xa0юго-запада США и\xa0из\xa0близлежащих регионов Мексики. Разделение двух групп, вероятно, произошло около 34\xa0миллионов лет назад. Альтернативные гипотезы, согласно которым игуаны расселились на\xa0острова Тихого океана через Евразию, Австралию или Антарктиду, требуют слишком много допущений и\xa0не\xa0согласуются с\xa0палеонтологическими данными. Как отмечается в\xa0статье для журнала Proceedings of\xa0the National Academy of\xa0Sciences, представители многих групп животных, предположительно, расселялись по\xa0океану на\xa0растительных плотах; однако ни\xa0один из\xa0них не\xa0сове

In [55]:
df.to_csv('news_plus_1.csv', 
          index = False, 
          sep = ';',  #  Сюда можно поставить произвольный символ, чтоб новость не ломать
          encoding = 'utf-8-sig')

Теперь столбец со сложностью точно числовой. Можем даже построить для него гистограмму.

In [56]:
%matplotlib inline
df.complexity.plot.hist()

<Axes: ylabel='Frequency'>

Осталось только заменить непонятные символы `\xa0` на пробелы:

In [57]:
df['clean_text'] = [t.replace("\xa0", " ") for t in df.full_text]

In [58]:
df.clean_text[0]

'Приматологи из Австралии, Германии и США обнаружили, что только взрослые приматы лучше запоминают предметы, с которыми взаимодействует человек, а не механическая клешня. Этот эффект они объяснили повышенным вниманием к социальному контексту, на который указывала пониженная частота сердечных сокращений. Исследование опубликовано в журнале Animal Behaviour.\n2 апреля отмечается Всемирный день информирования об аутизме, поэтому сегодня новости на N + 1 выходят с иллюстрациями от студентов «Антон тут рядом». Этот благотворительный фонд уже больше десяти лет помогает людям с расстройствами аутистического спектра в России\nЖизнь в группах требует особой чувствительности к социальным стимулам — и многие приматы обладают такой чувствительностью. Они отличают лица от других стимулов уже с рождения, а во взрослом возрасте быстрее идентифицируют и обрабатывают социальные стимулы. Иногда обезьяны даже отказываются от вознаграждения, чтобы смотреть на фото или видео с сородичами вместо фото и виде

Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [59]:
df.to_excel('nplus-news.xlsx')